<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/MicroVideoRecommandationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install py4j

     |████████████████████████████████| 204kB 2.8MB/s 


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark.sql import SparkSession


In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df =  spark.read.csv("/content/gdrive/My Drive/Major project Dataset/MicroVideoData.csv", inferSchema=True, header=True)

In [10]:
df.select('v_title').count()

4691

In [11]:
df.collect()[50]

Row(_c0=None, v_id=None, v_title=None, viewCount=None, likeCount=None, dislikeCount=None, favoriteCount=None, commentCount=None)

In [0]:
df=df.filter(df.v_id != 'v_id')

In [13]:
df.collect()[50]

Row(_c0=0, v_id='XJdOh5egYNo', v_title='Instagram Pe DM kardiya h mene ft. AJ |Oye It’s Prank | Oye It’s Uncut', viewCount=459141, likeCount=25490, dislikeCount=555, favoriteCount=0, commentCount=450)

In [14]:
df.collect()

[Row(_c0=0, v_id='BFmsUeu-u48', v_title='TRY NOT TO LAUGH - Funny Fails..Gravity Always Wins!', viewCount=764960, likeCount=5514, dislikeCount=676, favoriteCount=0, commentCount=190),
 Row(_c0=1, v_id='jv5RtGHKLWg', v_title='Top 10 Pranks of 2019 | Best of Just For Laughs Gags', viewCount=4422051, likeCount=45366, dislikeCount=2320, favoriteCount=0, commentCount=1064),
 Row(_c0=2, v_id='FFLTU9eIijw', v_title='TRY NOT TO LAUGH 😂🔥😹', viewCount=16364549, likeCount=139697, dislikeCount=8530, favoriteCount=0, commentCount=3283),
 Row(_c0=3, v_id='nuroWODcgL4', v_title='Indian StandUp Comedian ❖ Live comedy shows ❖ Canvas Laugh Club ❖Comedy shows in india ❖Sujit Pandey', viewCount=442566, likeCount=8672, dislikeCount=433, favoriteCount=0, commentCount=229),
 Row(_c0=4, v_id='Rv0IvHA1zlI', v_title='▶ BEST Just For Laughs Gags 2019 | Compilation Pranks BEST [#01]', viewCount=11329698, likeCount=39229, dislikeCount=4697, favoriteCount=0, commentCount=761),
 Row(_c0=5, v_id='e6rUXtZwf7U', v_titl

In [0]:
titleRDD = df.select('v_title').rdd.flatMap(lambda x:x)

In [16]:
titleRDD.collect()

['TRY NOT TO LAUGH - Funny Fails..Gravity Always Wins!',
 'Top 10 Pranks of 2019 | Best of Just For Laughs Gags',
 'TRY NOT TO LAUGH 😂🔥😹',
 'Indian StandUp Comedian ❖ Live comedy shows ❖ Canvas Laugh Club ❖Comedy shows in india ❖Sujit Pandey',
 '▶ BEST Just For Laughs Gags 2019 | Compilation Pranks BEST [#01]',
 'TRY NOT TO LAUGH - The best funny viral videos of the week!',
 'TRY NOT TO LAUGH AT THIS STUPID GAME! ft. Cray, Bazz &amp; Fasffy',
 '▶ BEST 2019 Just For Laughs Gags | NEW Compilation [#55]',
 'Nicolle Wallace: &#39;I don&#39;t Know Whether To Cry Or Laugh At McConnell&#39;s Political Stupidity&#39; | MSNBC',
 'New Top Funny Comedy Video Try Not To Laugh',
 'TRY NOT TO LAUGH | Funny Fails Videos 2019 | People Don&#39;t Know What They&#39;re Doing',
 'Top New Comedy Video 2019 | Try To Not Laugh | Episode-36 | By Fun ki vines',
 'Best of Just For Laughs Gags Compilation 2019 - Funnny Pranks Videos 2019',
 'Try Not To Laugh - Funny Videos 2019 - There She Goes',
 'Try Not To La

In [17]:
pip install langdetect

     |████████████████████████████████| 1.0MB 2.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=e76d406a2523b8923ec7c68916f234bd3082924f40f01d7fd3642fe3f2d279c8
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
from langdetect import detect
import pandas as pd

In [19]:
(pd.DataFrame(pd.Series('This is a string'))[0].apply(detect)=='en').bool()

True

In [0]:
def remove_langs(x):
  if((pd.DataFrame(pd.Series(x))[0].apply(detect)=='en').bool()):
    return x
  pass
  
titleRDD1=titleRDD.map(remove_langs)

In [0]:
titleRDD=titleRDD1.filter(lambda x:x is not None)

In [22]:
titleRDD.count()


3367

In [0]:
header = titleRDD.first()

In [0]:
data_rmv_col = titleRDD.filter(lambda row: row != header)

In [0]:
def lower(x):
  return x.lower()

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lower)

In [27]:
lowerCase_sentRDD.take(5)

['top 10 pranks of 2019 | best of just for laughs gags',
 'indian standup comedian ❖ live comedy shows ❖ canvas laugh club ❖comedy shows in india ❖sujit pandey',
 '▶ best just for laughs gags 2019 | compilation pranks best [#01]',
 'try not to laugh - the best funny viral videos of the week!',
 'try not to laugh at this stupid game! ft. cray, bazz &amp; fasffy']

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    #return doc
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [0]:
tagList=['prank','laugh','comedy','standup','funny','plitical','dog','cat','talent','girlfriend','cure','medicine','boyfroiend','boy','girl','kid','nepal','awesome','tom','lennon'
        ,'humor','instagram','facebook','twitter','youtube','google','police','couple','politics','pakistan','ghost','cute','scary','story','chocolate','challange','kite','pubg','corn'
        ]

In [30]:
sentenceTokenizeRDD.collect()

['prank good laugh gag',
 'indian standup comedian live comedy show canvas laugh club comedy show india sujit pandey',
 'best laugh gag compilation prank best',
 'try laugh good funny viral video week',
 'try laugh stupid game ft cray bazz amp fasffy',
 'best laugh gag new compilation',
 'nicolle wallace know cry laugh political msnbc',
 'new funny comedy video try laugh',
 'try laugh funny fail video people know',
 'new comedy video try laugh fun ki vine',
 'good laugh gag compilation funnny prank video',
 'try laugh funny video go',
 'try laugh challenge vine compilation good afv fail vine',
 'try laugh funny fail video compilation',
 'try watch laugh grin',
 'new comedy video try laugh episode maha fun tv',
 'try laugh funny video best clean vine compilation',
 'good funny video compilation december laugh',
 'try laugh funny video big',
 'try laugh funny video best clean vine week',
 'try laugh funny video best clean vine week',
 'try laugh funniest fail watch today',
 'try laugh fu